In [21]:
import pandas as pd
from docx import Document
import regex

In [1]:
document = Document("PMC8078050/PMC8078050_mmc1.docx")

NameError: name 'Document' is not defined

In [26]:
document.tables

In [27]:
for table in document.tables:
    data = [[cell.text for cell in row.cells] for row in table.rows]
    df = pd.DataFrame(data)
    print(df)

                                                    0                   1  \
0                           Predicted miRNA sequences       Name of miRNA   
1   Query  7   GGTGCCTGGAATATT  21\n           |||...  SARS-CoV-2-miR-005   
2   Query  5    TGGCTTTGTGTGCTGAC  21\n           ...  SARS-CoV-2-miR-007   
3   Query  2    AATAACAGTTACAC  15\n            ||...  SARS-CoV-2-miR-027   
4   Query  2   TGCAAAAGTATTCTAC  17\n           ||...  SARS-CoV-2-miR-029   
5   Query  1   CAAAGTTTTTATGACAGCATCT  22\n       ...  SARS-CoV-2-miR-055   
6   Query  1   CAAAGTTTTTATGACAGCATCT  22\n       ...  SARS-CoV-2-miR-055   
7   Query  1   GTTGGCACTTTTCTCAAAGCTT  22\n       ...  SARS-CoV-2-miR-058   
8   Query  1   TCTTAAAAGAGGGTG  15\n           |||...  SARS-CoV-2-miR-060   
9   Query  5    TTTTCTAAAACCACT  19\n            |...  SARS-CoV-2-miR-063   
10  Query  5   TGCTTTTCCATCATGA  20\n           ||...  SARS-CoV-2-miR-077   
11  Query  1   TTGTTGTTGGCCTTTACCAGAC  22\n       ...  SARS-CoV-2-miR-084   

In [45]:

def getNameSeqColumns(df):

    potentialColumns = list(df.loc[0,])
    potentialColumns = [x.strip() for x in potentialColumns]
    #print(potentialColumns)
    
    mirName = None
    mirSeq = None

    for xi, x in enumerate(potentialColumns):
        if "NAME" in x.upper() and mirName is None:
            mirName = xi

        elif "MIRNA SEQUENCE" in x.upper() and mirSeq is None:
            mirSeq = xi

    #print("Name:", mirName, potentialColumns[mirName])
    #print("Seq: ", mirSeq, potentialColumns[mirSeq])

    return mirName, mirSeq

In [30]:
def is_empty(elem):
    if elem is None:
        return True

    if len(str(elem)) == 0:
        return True

    return False

In [32]:
document.tables

In [46]:
fastaSequenceMirRE = regex.compile(r'^[ATCGU]{15,30}$')

for table in document.tables:
    data = [[cell.text for cell in row.cells] for row in table.rows]
    df = pd.DataFrame(data)

    mirName, mirSeq = getNameSeqColumns(df)
    
    if mirName == None or mirSeq == None:
        print("Skipping")
        break

    for ri, row in df.iterrows():

        if not is_empty(row[mirName]) and not is_empty(row[mirSeq]):

            fastaSequenceMir = fastaSequenceMirRE.match(str(row[mirSeq]))

            if fastaSequenceMir:
                print(row[mirName], row[mirSeq])